In [1]:
#set of cities
cities = {'Ams', 'Rot', 'Hag', 'Utr', 'Ein', 'Til', 'Gro', 'Alm', 'Bre', 'Nij'}

## start and end
start = 'Ams'
# e < u < l
## Distances between cities
d = {('Ams', 'Ams'): 0, ('Ams', 'Rot'): 58, ('Ams', 'Hag'): 61, ('Ams', 'Utr'): 53, ('Ams', 'Ein'): 131,
 ('Ams', 'Til'): 119,
 ('Ams', 'Gro'): 183,
 ('Ams', 'Alm'): 34,
 ('Ams', 'Bre'): 112,
 ('Ams', 'Nij'): 123,
 ('Rot', 'Ams'): 58,
 ('Rot', 'Rot'): 0,
 ('Rot', 'Hag'): 27,
 ('Rot', 'Utr'): 62,
 ('Rot', 'Ein'): 111,
 ('Rot', 'Til'): 77,
 ('Rot', 'Gro'): 246,
 ('Rot', 'Alm'): 100,
 ('Rot', 'Bre'): 51,
 ('Rot', 'Nij'): 109, ('Hag', 'Ams'): 61, ('Hag', 'Rot'): 27,
 ('Hag', 'Hag'): 0,
 ('Hag', 'Utr'): 70,
 ('Hag', 'Ein'): 143,
 ('Hag', 'Til'): 106,
 ('Hag', 'Gro'): 237,
 ('Hag', 'Alm'): 88,
 ('Hag', 'Bre'): 79,
 ('Hag', 'Nij'): 136,
 ('Utr', 'Ams'): 53,
 ('Utr', 'Rot'): 62,
 ('Utr', 'Hag'): 70,
 ('Utr', 'Utr'): 0,
 ('Utr', 'Ein'): 92,
 ('Utr', 'Til'): 80,
 ('Utr', 'Gro'): 188,
 ('Utr', 'Alm'): 42,
 ('Utr', 'Bre'): 73,
 ('Utr', 'Nij'): 90,
 ('Ein', 'Ams'): 131,
 ('Ein', 'Rot'): 111,
 ('Ein', 'Hag'): 143,
 ('Ein', 'Utr'): 92,
 ('Ein', 'Ein'): 0,
 ('Ein', 'Til'): 39,
 ('Ein', 'Gro'): 253,
 ('Ein', 'Alm'): 128,
 ('Ein', 'Bre'): 60,
 ('Ein', 'Nij'): 66,
 ('Til', 'Ams'): 119,
 ('Til', 'Rot'): 77,
 ('Til', 'Hag'): 106,
 ('Til', 'Utr'): 80,
 ('Til', 'Ein'): 39,
 ('Til', 'Til'): 0,
 ('Til', 'Gro'): 262,
 ('Til', 'Alm'): 116,
 ('Til', 'Bre'): 30,
 ('Til', 'Nij'): 80,
 ('Gro', 'Ams'): 183,
 ('Gro', 'Rot'): 246,
 ('Gro', 'Hag'): 237,
 ('Gro', 'Utr'): 188,
 ('Gro', 'Ein'): 253,
 ('Gro', 'Til'): 262,
 ('Gro', 'Gro'): 0,
 ('Gro', 'Alm'): 155,
 ('Gro', 'Bre'): 255,
 ('Gro', 'Nij'): 208,
 ('Alm', 'Ams'): 34,
 ('Alm', 'Rot'): 100,
 ('Alm', 'Hag'): 88,
 ('Alm', 'Utr'): 42,
 ('Alm', 'Ein'): 128,
 ('Alm', 'Til'): 116,
 ('Alm', 'Gro'): 155,
 ('Alm', 'Alm'): 0,
 ('Alm', 'Bre'): 109,
 ('Alm', 'Nij'): 112,
 ('Bre', 'Ams'): 112,
 ('Bre', 'Rot'): 51,
 ('Bre', 'Hag'): 79,
 ('Bre', 'Utr'): 73,
 ('Bre', 'Ein'): 60,
 ('Bre', 'Til'): 30,
 ('Bre', 'Gro'): 255,
 ('Bre', 'Alm'): 109,
 ('Bre', 'Bre'): 0,
 ('Bre', 'Nij'): 96,
 ('Nij', 'Ams'): 123,
 ('Nij', 'Rot'): 109,
 ('Nij', 'Hag'): 136,
 ('Nij', 'Utr'): 90,
 ('Nij', 'Ein'): 66,
 ('Nij', 'Til'): 80,
 ('Nij', 'Gro'): 208,
 ('Nij', 'Alm'): 112,
 ('Nij', 'Bre'): 96,
 ('Nij', 'Nij'): 0}

In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [12]:
model = pe.ConcreteModel()

In [13]:
model.N = pe.Set(initialize =cities ,ordered = False)

In [14]:
model.x = pe.Var(model.N, model.N, domain = pe.Binary)
model.u = pe.Var(model.N, domain = pe.NonNegativeIntegers)

In [15]:
objExpr = sum(model.x[i,j] * d[i,j] for i in model.N for j in model.N)
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [16]:
# Tour leaves each city
model.cnstrOut = pe.ConstraintList()
for i in model.N: 
    expression = sum(model.x[i,j] for j in model.N) == 1
    model.cnstrOut.add(expression)
    
# Tour enters each city
model.cnstrIn = pe.ConstraintList()
for j in model.N: 
    expression = sum(model.x[i,j] for i in model.N) == 1
    model.cnstrIn.add(expression)

# Subtours elimination constraints
model.cnstrSubTour = pe.ConstraintList()
for i in model.N:
    for j in model.N:
        if i != j and j != start:
            expression = model.u[i] + 1 <= model.u[j] + len(model.N)*(1 - model.x[i,j])
            model.cnstrSubTour.add(expression)

## Starting point
startExpr = model.u[start] == 1
model.cnstrStart = pe.Constraint(expr = startExpr)

## Prevent loops on a singe nodes
model.cnstrSingleLoop = pe.ConstraintList()
for i in model.N:
    expression = model.x[i,i] == 0
    model.cnstrSingleLoop.add(expression)

  


In [17]:
#model.pprint()
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-14
Read LP format model from file C:\Users\timva\AppData\Local\Temp\tmppdwmd051.pyomo.lp
Reading time = 0.01 seconds
x111: 113 rows, 111 columns, 455 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 113 rows, 111 columns and 455 nonzeros
Model fingerprint: 0x34808f2a
Variable types: 1 continuous, 110 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Found heuristic solution: objective 1107.0000000
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolved: 101 rows, 99 columns, 414 nonzeros
Variable types: 0 continuous, 99 integer (90 binary)

Root relaxation: objective 6.760000e+02, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    

In [18]:
print(result.solver.status)
print(result.solver.termination_condition)

ok
optimal


In [19]:
print("Objective value = "+ str(pe.value(model.obj)))

Objective value = 732.0


In [20]:
print("Tour:\n")
for i in model.N:
    for j in model.N:
        if(pe.value(model.x[i,j]) > 0):
            print(str(model.x[i,j]) + " = " + str(pe.value(model.x[i,j])))

print("\nStarting time:\n")
for i in model.N:
        print(str(model.u[i]) + " = " + str(pe.value(model.u[i])))
            


Tour:

x[Rot,Hag] = 1.0
x[Til,Bre] = 1.0
x[Nij,Ein] = 1.0
x[Alm,Gro] = 1.0
x[Ein,Til] = 1.0
x[Bre,Rot] = 1.0
x[Gro,Nij] = 1.0
x[Hag,Ams] = 1.0
x[Utr,Alm] = 1.0
x[Ams,Utr] = 1.0

Starting time:

u[Rot] = 9.0
u[Til] = 7.0
u[Nij] = 5.0
u[Alm] = 3.0
u[Ein] = 6.0
u[Bre] = 8.0
u[Gro] = 4.0
u[Hag] = 10.0
u[Utr] = 2.0
u[Ams] = 1.0
